# ChIP-seq intro 3
Huitian Diao
* __References__: 
* [ENCODE ChIP-seq pipeline](https://www.encodeproject.org/pipelines/ENCPL138KID/)

## 1. Filter blacklisted regions
### 1.1 Download blacklisted regions
* [Chip-blacklisted regions](https://sites.google.com/site/anshulkundaje/projects/blacklists)

### 1.2 Download aligned files
* [SRR3001750_srt_dupr.chr10.bam](https://drive.google.com/open?id=17ASjdXGqagqblAzk2V3WBBDbcuEYyXDY)

* Move download bam file and blacklist bed file into the new folder

### 1.3 Use bedtools to remove blacklisted regions (Optional)
* [Bioconda: bedtools](https://bioconda.github.io/recipes/bedtools/README.html)
* [bedtools - intersect](https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html)

In [3]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
bedtools intersect -abam SRR3001750_srt_dupr.chr10.bam -b mm10.blacklist.bed -v > SRR3001750_srt_dupr.chr10_flb.bam

In [7]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
ls -alh SRR3001750_srt_dupr*

-rw-r--r--@ 1 sfchen  staff    36M Nov 13 08:28 SRR3001750_srt_dupr.chr10.bam
-rw-r--r--  1 sfchen  staff    36M Nov 13 08:36 SRR3001750_srt_dupr.chr10_flb.bam
-rw-r--r--@ 1 sfchen  staff    32M Nov 13 08:28 SRR3001750_srt_dupr.chr12.bam
-rw-r--r--@ 1 sfchen  staff    98M Nov 13 08:28 SRR3001750_srt_dupr.chr2.chr11.bam


### 1.4 Use awk to filter chrM (Optional)

In [8]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
samtools view -h SRR3001750_srt_dupr.chr10_flb.bam | awk '{if ($3!="chrM") {print $0}}' > SRR3001750_srt_dupr.chr10_flb_flt.sam
samtools view -bS SRR3001750_srt_dupr.chr10_flb_flt.sam > SRR3001750_srt_dupr.chr10_flb_flt.bam
rm SRR3001750_srt_dupr.chr10_flb_flt.sam

## 2. Peak calling with MACS2
### 2.1 Peak calling

* [__MACS2 parameters__](https://github.com/taoliu/MACS)
* -t: Treatment filename
* -f: Format
* -g: Genome size
* -n: Name string of experiment
* -p: P-value cutoff
* --broad: Broad regions
* --nomodel: Bypass shifting model
* --bdg: Store the fragment pileup, control lambda, -log10pvalue and -log10qvalue scores in bedGraph files
* --shift: move 5' end towards 5' -> 3'
* --extsize: extend reads from 5' -> 3'
* --keep-dup: keep duplicate tags at exact same location

#### 2.1.1 Use MACS2 to estimate extension size

In [9]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3 
macs2 predictd -i SRR3001750_srt_dupr.chr10_flb_flt.bam &> SRR3001750_srt_dupr.chr10_flb_flt.predictd

#### 2.1.2 Peak calling for histone modification ChIP-seq
* [Guidline from MACS Github page](https://github.com/taoliu/MACS/wiki/Call-differential-binding-events)

In [10]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
macs2 callpeak -t SRR3001750_srt_dupr.chr10_flb_flt.bam -f BAM -n SRR3001750_srt_dupr.chr10 -g mm -p 1e-2 --broad --nomodel --bdg --shift 0 --extsize 147 --keep-dup all 

INFO  @ Tue, 13 Nov 2018 08:40:06: 
# Command line: callpeak -t SRR3001750_srt_dupr.chr10_flb_flt.bam -f BAM -n SRR3001750_srt_dupr.chr10 -g mm -p 1e-2 --broad --nomodel --bdg --shift 0 --extsize 147 --keep-dup all
# ARGUMENTS LIST:
# name = SRR3001750_srt_dupr.chr10
# format = BAM
# ChIP-seq file = ['SRR3001750_srt_dupr.chr10_flb_flt.bam']
# control file = None
# effective genome size = 1.87e+09
# band width = 300
# model fold = [5, 50]
# pvalue cutoff for narrow/strong regions = 1.00e-02
# pvalue cutoff for broad/weak regions = 1.00e-01
# qvalue will not be calculated and reported as -1 in the final output.
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is on
# Paired-End mode is off
 
INFO  @ Tue, 13 Nov 2018 08:40:06: #1 read tag file

### 2.2 Check output files
* NAME_peaks.broadPeak: BED6+4 format file which contains the peak locations together with peak summit, pvalue and qvalue.
* NAME_peaks.gappedPeak: BED12+3 format which contains both the broad region and narrow peaks. 
* NAME_treat_pileup.bdg: Fragment pileup, control lambda, -log10pvalue and -log10qvalue scores
* NAME_control_lambda.bdg: Local lambda values from control
* NAME_peaks.xls: A tabular file which contains information about called peaks. 

In [11]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
wc -l *

  161849 SRR3001750_srt_dupr.chr10.bam
 1201248 SRR3001750_srt_dupr.chr10_control_lambda.bdg
  162223 SRR3001750_srt_dupr.chr10_flb.bam
  149477 SRR3001750_srt_dupr.chr10_flb_flt.bam
      17 SRR3001750_srt_dupr.chr10_flb_flt.predictd
   95496 SRR3001750_srt_dupr.chr10_peaks.broadPeak
   95496 SRR3001750_srt_dupr.chr10_peaks.gappedPeak
   95520 SRR3001750_srt_dupr.chr10_peaks.xls
 1195664 SRR3001750_srt_dupr.chr10_treat_pileup.bdg
  148051 SRR3001750_srt_dupr.chr12.bam
  440113 SRR3001750_srt_dupr.chr2.chr11.bam
  329402 SRR3001751_srt_dupr.chr10.bam
  296321 SRR3001751_srt_dupr.chr12.bam
     164 mm10.blacklist.bed
      17 predictd
 4371058 total


In [12]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
head SRR3001750_srt_dupr.chr10_peaks.broadPeak

chr10	3099953	3100100	SRR3001750_srt_dupr.chr10_peak_1	5	.	1.57646	1.51967	0.50929
chr10	3100698	3101016	SRR3001750_srt_dupr.chr10_peak_2	6	.	1.92416	1.94794	0.66660
chr10	3101562	3102167	SRR3001750_srt_dupr.chr10_peak_3	7	.	2.07886	2.16806	0.76456
chr10	3102603	3102750	SRR3001750_srt_dupr.chr10_peak_4	4	.	1.45300	1.25693	0.46500
chr10	3103031	3103734	SRR3001750_srt_dupr.chr10_peak_5	5	.	1.60470	1.42813	0.52361
chr10	3109444	3110068	SRR3001750_srt_dupr.chr10_peak_6	34	.	2.17264	6.60279	3.42375
chr10	3110304	3110452	SRR3001750_srt_dupr.chr10_peak_7	13	.	1.71785	3.46310	1.36576
chr10	3111278	3114010	SRR3001750_srt_dupr.chr10_peak_8	21	.	1.76784	4.59360	2.13602
chr10	3114302	3115301	SRR3001750_srt_dupr.chr10_peak_9	13	.	1.63792	3.42048	1.37027
chr10	3116068	3117178	SRR3001750_srt_dupr.chr10_peak_10	44	.	2.28970	8.14642	4.40802


chr10	3099953	3100100	SRR3001750_srt_dupr.chr10_peak_1	5	.	1.57646	1.51967	0.50929
chr10	3100698	3103734	SRR3001750_srt_dupr.chr10_peak_2	6	.	1.82648	1.78537	0.63420
chr10	3109444	3110452	SRR3001750_srt_dupr.chr10_peak_3	29	.	2.07850	5.95290	2.99776
chr10	3111278	3115301	SRR3001750_srt_dupr.chr10_peak_4	20	.	1.74597	4.39613	2.00712
chr10	3116068	3117178	SRR3001750_srt_dupr.chr10_peak_5	44	.	2.28970	8.14641	4.40801
chr10	3124590	3125334	SRR3001750_srt_dupr.chr10_peak_6	5	.	1.66666	1.45416	0.52554
chr10	3126076	3128929	SRR3001750_srt_dupr.chr10_peak_7	5	.	1.64651	1.47578	0.53620
chr10	3129691	3130649	SRR3001750_srt_dupr.chr10_peak_8	4	.	1.39770	1.12225	0.42857
chr10	3131826	3131973	SRR3001750_srt_dupr.chr10_peak_9	4	.	1.34671	1.06692	0.42004
chr10	3132717	3132997	SRR3001750_srt_dupr.chr10_peak_10	4	.	1.38884	1.12172	0.43652

In [13]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
head SRR3001750_srt_dupr.chr10_peaks.gappedPeak

chr10	3099953	3100100	SRR3001750_srt_dupr.chr10_peak_1	5	.	3099953	3100100	0	2	1,1	0,146	1.57646	1.51967	0.50929
chr10	3100698	3101016	SRR3001750_srt_dupr.chr10_peak_2	6	.	3100698	3101016	0	2	1,1	0,317	1.92416	1.94794	0.66660
chr10	3101562	3102167	SRR3001750_srt_dupr.chr10_peak_3	7	.	3101562	3102167	0	3	1,273,1	0,74,604	2.07886	2.16806	0.76456
chr10	3102603	3102750	SRR3001750_srt_dupr.chr10_peak_4	4	.	3102603	3102750	0	2	1,1	0,146	1.45300	1.25693	0.46500
chr10	3103031	3103734	SRR3001750_srt_dupr.chr10_peak_5	5	.	3103031	3103734	0	2	1,1	0,702	1.60470	1.42813	0.52361
chr10	3109444	3110068	SRR3001750_srt_dupr.chr10_peak_6	34	.	3109444	3110068	0	4	1,344,155,1	0,13,457,623	2.17264	6.60279	3.42375
chr10	3110304	3110452	SRR3001750_srt_dupr.chr10_peak_7	13	.	3110304	3110452	0	2	1,1	0,147	1.71785	3.46310	1.36576
chr10	3111278	3114010	SRR3001750_srt_dupr.chr10_peak_8	21	.	3111278	3114010	0	5	1,1696,183,509,1	0,13,1837,2213,2731	1.76784	4.59360	2.13602
chr10	3114302	3115301	SRR3001750_srt_dupr.ch

In [14]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
head SRR3001750_srt_dupr.chr10_treat_pileup.bdg

chr10	0	3099953	0.00000
chr10	3099953	3100100	1.00000
chr10	3100100	3100698	0.00000
chr10	3100698	3100845	1.00000
chr10	3100845	3100867	0.00000
chr10	3100867	3100869	1.00000
chr10	3100869	3101014	2.00000
chr10	3101014	3101016	1.00000
chr10	3101016	3101562	0.00000
chr10	3101562	3101636	1.00000


#### Convert bedgraph to bw
* [bedGraphToBigWig usage](https://genome.ucsc.edu/goldenpath/help/bigWig.html)
* [mm10.chrom.sizes](http://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes)

In [16]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
bedGraphToBigWig SRR3001750_srt_dupr.chr10_treat_pileup.bdg mm10.chrom.sizes SRR3001750_srt_dupr.chr10_treat_pileup.bw 

In [18]:
%%bash
cd /Users/sfchen/Documents/2018_Su/ChIP-seq_3
ls -lh

total 1071144
-rw-r--r--@ 1 sfchen  staff    36M Nov 13 08:28 SRR3001750_srt_dupr.chr10.bam
-rw-r--r--  1 sfchen  staff    37M Nov 13 08:40 SRR3001750_srt_dupr.chr10_control_lambda.bdg
-rw-r--r--  1 sfchen  staff    36M Nov 13 08:36 SRR3001750_srt_dupr.chr10_flb.bam
-rw-r--r--  1 sfchen  staff    36M Nov 13 08:39 SRR3001750_srt_dupr.chr10_flb_flt.bam
-rw-r--r--  1 sfchen  staff   1.1K Nov 13 08:39 SRR3001750_srt_dupr.chr10_flb_flt.predictd
-rw-r--r--  1 sfchen  staff   8.1M Nov 13 08:40 SRR3001750_srt_dupr.chr10_peaks.broadPeak
-rw-r--r--  1 sfchen  staff    11M Nov 13 08:40 SRR3001750_srt_dupr.chr10_peaks.gappedPeak
-rw-r--r--  1 sfchen  staff   8.6M Nov 13 08:40 SRR3001750_srt_dupr.chr10_peaks.xls
-rw-r--r--  1 sfchen  staff    37M Nov 13 08:40 SRR3001750_srt_dupr.chr10_treat_pileup.bdg
-rw-r--r--  1 sfchen  staff   5.4M Nov 13 08:45 SRR3001750_srt_dupr.chr10_treat_pileup.bw
-rw-r--r--@ 1 sfchen  staff    32M Nov 13 08:28 SRR3001750_srt_dupr.chr12.bam
-rw-r--r--@ 1 sfchen  staff    9